In [ ]:
def selectRows():
    import sqlite3
    con = sqlite3.connect('../data/GuidePod.sqlite')
    cur = con.cursor()
    all_pods = []
    try:
        sql = '''select distinct id, country, FeedURL  from podcast_master pd  ''' + \
                '''where feedURL is not null AND rank <= 0  ''' + \
                '''AND id not in (select DISTINCT id from podcast_reviews) '''
        rows = cur.execute(sql)
    except Exception as err:
        print (err)
        pass
    #con.commit()
    for row in rows:
        all_pods.append([row[0], row[1]])

    cur.close()
    con.close()
    
    return all_pods


In [ ]:
qq = selectRows()
print (len(qq))

In [ ]:
import json
from urllib.request import urlopen, Request,FancyURLopener
from urllib import error
import urllib
import csv
import re

#https://itunes.apple.com/us/rss/customerreviews/id=1468964257/json  - This URL gets only the first page which has 50 reviews

#Use this URL to find more pages of review.  By altering page=1 variable, to say page=2 you can get the 2nd page of 50 reviews
#
#findreviews = "https://itunes.apple.com/us/rss/customerreviews/page=1/id=" + qq + "/sortby=mostrecent/json"
#

def findReviews(pageNum , podcastList):
    rows = []
    breaknow = 0
    for i in podcastList: 
        country_code = i[1]
        podcast_id = i[0]
        rv = "https://itunes.apple.com/" + \
                country_code + "/rss/customerreviews/page=" + \
                str(pageNum) +"/id="+ str(podcast_id) +"/sortby=mostrecent/json"
        #print (rv)
        try:
            with  urlopen(rv) as data_file:    
                reviewdata = json.load(data_file)
                reviewdatafeed = reviewdata["feed"] 

                try:
                    qq = reviewdatafeed["entry"]
                    for eachitem in qq:
                        rvwHeading=re.sub(r'[^A-Za-z0-9 \.]+', '', str(eachitem["title"]["label"])) 
                        rvwText= re.sub(r'[^A-Za-z0-9 \.]+', '',str(eachitem["content"]["label"]))
                        rvwRating = str(eachitem["im:rating"]["label"])
                        x = [podcast_id, country_code, rvwHeading, rvwText,rvwRating]
                        rows.append(x)
                except Exception as err:
                    pass
        except urllib.error.HTTPError as err1:
            print (rv, err1)
            breaknow = 1
            break
        except Exception as err:
            print(err)
            pass
        
        if breaknow == 1:
            break
        
        return rows

In [ ]:
import sqlite3
import time

con = sqlite3.connect('../data/GuidePod.sqlite')
cur = con.cursor()

iteration = 0

for eachpod in mm:
    jj = eachpod
    for pgNum in range(1,2):
        iteration += 1
        if iteration % 100 == 0: 
            time.sleep(0.5) 
            
        rvws = findReviews(pgNum, [jj])
        if rvws == None:
            continue
        for x in rvws:
            try:
                cur.execute('''INSERT INTO podcast_reviews_temp values (?,?,?,?,?, ?)''',(x[0],x[1],x[2],x[3],x[4], pgNum))
            except Exception as err:
                print (err)
                pass
        con.commit()
con.close()